# Interactivity

In [31]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from ipywidgets import widgets, Layout
import folium
from IPython.display import clear_output

# Get and Collapse Data by State, County

In [32]:
# Get cleaned gun violence master dataset and states data
directory = os.path.dirname(str(Path().resolve()) + '\\')
path = os.path.join(directory, 'data', 'final_data.csv')
gunViolence = pd.read_csv(path)
gunViolence.fillna(0, inplace=True)  # Fill in nans w/ zeros

In [33]:
stateCols = ['state', 'Mental_Health_Records_Submitted_2008', 'Mental_Health_Records_Submitted_2017', 'Gun_Sale_Denials_2008', 
             'Gun_Sale_Denials_2017', 'Giffords Gun Safety Score',  'Gun Deaths per 100k People (2018)', 
             'Gun Death Rate (Ranked High to Low)', '# of guns per capita', '# of guns registered', 'Permit Type',  
             'Handgun_Carry_Permit_Fee', 'Years_Valid', '5_Year_Cost', 'Happiness Score']

stateData = gunViolence[stateCols].drop_duplicates('state')

# Display Interactive Map with Data by State (Most Recent Code)

In [34]:
# embed_map function written by user: ocefpaf to patch folium rendering issue with > 80 objects in Google Chrome
def embed_map(m):
    from IPython.display import IFrame
    m.save('index.html')
    return IFrame('index.html', width='100%', height='650px')

# Define list of factors that should be plotted with high values as red instead of green
badFactorsList = ['Gun_Death_Rate_2018 (per 100k people)', '# of guns per capita', '# of guns registered',]
def getColor(col):
    if (col in badFactorsList):
        return 'OrRd'
    else:
        return 'YlGn'

In [35]:
# Create interactive widgets with mapping function
dropdown = widgets.Dropdown(
    options=stateData.columns.values[1:],
    value='5_Year_Cost',
    description='Factor to Plot:',
    disabled=False,
    layout=Layout(width='40%'),
    style={'description_width': 'initial'}
)

slider = widgets.IntSlider(min=0,max=5000,step=100,value=100,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
button = widgets.Button(description = "Go!")
display(dropdown)
display(slider)
display(button)

def on_button_clicked(b):
    clear_output()
    display(dropdown)
    display(slider)
    display(button)
    folium_map = folium.Map(location=[35, -97], zoom_start=4, tiles="Mapbox Bright")   

    folium.Choropleth(
        geo_data='us_states_20m.json',
        name=dropdown.value,
        data=stateData,
        columns=['state', dropdown.value],
        key_on='feature.properties.NAME',
        fill_color=getColor(dropdown.value),
        fill_opacity=0.5,
        line_opacity=0.2,
        legend_name=dropdown.value
    ).add_to(folium_map)
    
    folium.LayerControl().add_to(folium_map)
    
    for i, row in gunViolence.sample(frac=1).head(slider.value).iterrows():
        marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], radius = (row['n_killed'] + row['n_injured']))
        marker.add_to(folium_map)
    toDisp = embed_map(folium_map)
    display(toDisp)

button.on_click(on_button_clicked)

Dropdown(description='Factor to Plot:', index=5, layout=Layout(width='40%'), options=('Mental_Health_Records_S…

IntSlider(value=100, description='Number of Incidents:', layout=Layout(width='40%'), max=5000, step=100, style…

Button(description='Go!', style=ButtonStyle())

# Subcomponent Demonstration Code

In [42]:
# Simple Overlay Demonstration (no Data)

def embed_map(m):
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

folium_map = folium.Map(location=[35, -97],
                        zoom_start=4,
                        tiles="Mapbox Bright")
folium.GeoJson('us_counties_20m.json', name='US Counties').add_to(folium_map)
folium.GeoJson('us_states_20m.json', name='US States').add_to(folium_map)
folium.LayerControl().add_to(folium_map)
embed_map(folium_map)

In [41]:
# Widget Demonstration Code

button = widgets.Button(description = "Go!")
slider = widgets.IntSlider(min=0,max=5000,step=100,value=100,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
display(slider)
display(button)

def on_button_clicked(b):
    clear_output()
    display(slider)
    display(button)
    folium_map = folium.Map(location=[35, -97],
                            zoom_start=4,
                            tiles="Mapbox Bright")
    for i, row in gunViolence.head(slider.value).iterrows():
        marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], radius = (row['n_killed'] + row['n_injured']))
        marker.add_to(folium_map)
    display(folium_map)

button.on_click(on_button_clicked)

IntSlider(value=100, description='Number of Incidents:', layout=Layout(width='40%'), max=5000, step=100, style…

Button(description='Go!', style=ButtonStyle())